In [ ]:
%run ../pathutils.ipynb
%run ../database.ipynb
%run ../export.ipynb
%run health.ipynb
%run database.ipynb
%run utils.ipynb

In [ ]:
# Construct the query file for the database growth data
query = construct_query("weather", "db-growth.sql", {})

# Run the query to retrieve the data
df = query_data("weather", query)

# Convert column titles to lowercase
df.columns = df.columns.str.lower()

# Convert the timestamp string to a date and time - need to remove the trailing Z or it won't parse
df["timestamp"] = (
    df["timestamp"]
    .str.rstrip("Z")
    .pipe(pd.to_datetime, utc=True)
)

In [ ]:

# Strip the timezone from the timestamp, as this will cause the export to spreadsheet to fail (Excel can't
# handle dates with timezone information)
df.set_index("timestamp", inplace=True)
df.index = df.index.tz_localize(None)

# Extract a dataframe containing only table data
tables_df = df[df.object_type == "table"]

# Create a pivot with the table names as headers and each row containing the data
# for one snapshot
tables_pivot = tables_df.pivot_table(
    index="timestamp",
    columns="object_name",
    values="bytes",
    aggfunc="sum"
).sort_index()

display(tables_pivot)

# Resample the pivot to show differences as the values rather than size
growth_rates = (
    tables_pivot
    .diff()
    .resample("1D")
    .sum()
    / 1024 / 1024
)

In [ ]:
# Get the export folder path
export_folder_path = get_export_folder_path("management")

# Export the data to a spreadsheet
export_to_spreadsheet(export_folder_path, "database_size.xlsx", {
    "Snapshots": df,
    "Table Size": tables_pivot,
    "Table Growth": growth_rates
})

export_to_spreadsheet(export_folder_path, "table_size.xlsx", {
    "Table Size": tables_pivot
})

export_to_spreadsheet(export_folder_path, "table_growth.xlsx", {
    "Table Growth": growth_rates
})

In [ ]:
import matplotlib.colors as mcolors
import numpy as np

def pastelize(color, amount=0.15):
    """
    Blend a color with white. Amount ranges from 0 (original colour) to 1.0 (white)
    """
    c = np.array(mcolors.to_rgb(color))
    return tuple(c + (1 - c) * amount)

## Database Size

In [ ]:
import matplotlib.pyplot as plt

db = df[df.object_type == "db"].sort_index()

plt.figure(figsize=(12, 4))
plt.plot(db.index, db.bytes / 1024 / 1024)
plt.xlabel("Date")
plt.ylabel("Database size (MB)")
plt.title("Weather Station Database Growth")
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "database_size", "png")

plt.show()

## Individual Table Sizes

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Create a colour map
base = plt.cm.tab20(np.linspace(0, 1, len(tables_pivot.columns)))
colors = [pastelize(c, amount=0.15) for c in base]

plt.stackplot(
    tables_pivot.index,
    [tables_pivot[c] / 1024 / 1024 for c in tables_pivot.columns],
    labels=tables_pivot.columns,
    colors=colors
)

plt.xlabel("Date")
plt.ylabel("Size (MB)")
plt.title("Table Sizes")

# Legend underneath the plot
plt.legend(
    loc="upper center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=min(len(tables_pivot.columns), 4),
    fontsize="small",
    frameon=False
)

plt.tight_layout()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "table_size", "png")

plt.show()

## Individual Table Growth

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Plot the data for each table
for table in tables_pivot.columns:
    plt.plot(tables_pivot.index, tables_pivot[table] / 1024 / 1024, label=table)

# Legend underneath the plot
plt.legend(
    loc="upper center",
    bbox_to_anchor=(0.5, -0.15),
    ncol=min(len(tables_pivot.columns), 4),
    fontsize="small",
    frameon=False
)

plt.ylabel("Growth (MB)")
plt.title("Table Growth")
plt.tight_layout()

# Export to PNG or PDF, if required
export_chart(export_folder_path, "table_growth", "png")

plt.show()